In [19]:
from typing import TypedDict, Annotated, List, Dict
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_openai.chat_models.base import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

from dotenv import load_dotenv
from langsmith import Client
import json

import sqlite3
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool

load_dotenv() 
client = Client() # langsmith 추적

class RealEstateState(TypedDict): # 그래프의 상태를 정의하는 클래스
    real_estate_type: Annotated[str ,"부동산 유형 (예: 아파트, 상가)"]
    keywordlist: Annotated[List[Dict] ,"키워드 리스트"]
    messages: Annotated[list, add_messages]
    query_sql: Annotated[str ,"생성된 SQL 쿼리"]
    results: Annotated[List[Dict], "쿼리 결과"]
    answers: Annotated[List[str], "최종 답변 결과"]
    query_answer:Annotated[str, 'answer다듬기']

# Step 1: LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=1)

# Step 2: StateGraph 정의
workflow = StateGraph(RealEstateState)

In [20]:
def filter_node(state:RealEstateState) -> RealEstateState:
    system_prompt = """
    Classify if a given question is related to real estate. If the question is related to topics such as property transactions, rental conditions, location recommendations, or property features, return Pass. If it's not directly related to real estate, return Fail.

    # Output Format
    - Return Pass if the question is real estate-related, otherwise return Fail.

    # Examples
    - 입력: '서울 아파트 매매 가격이 어떻게 되나요?'
      출력: Pass

    - 입력: '이 동네 전세 시세 알려주세요'
      출력: Pass

    - 입력: '서울에서 월세 계약 조건이 어떻게 되나요?'
      출력: Pass

    - 입력: '서울 아파트 매매가 얼마인가요?'
      출력: Pass

    - 입력: '대치동에서 버스정류장과 지하철이 가장 가까운 곳으로 알려줘'
      출력: Pass

    - 입력: '여자 혼자 살기 좋은 곳 추천해줘'
      출력: Pass

    - 입력: '교통이 편리하고 저렴한 원룸 추천해줘'
      출력: Pass

    - 입력: '이 음식점이 맛있나요?'
      출력: Fail

    - 입력: '서울에서 가장 큰 공원이 어디인가요?'
      출력: Fail
    """
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(state["messages"][-1].content)
    ])

    real_estate_type = response.content.strip()
    return RealEstateState(real_estate_type=real_estate_type)

def fiter_router(state: RealEstateState):
    # This is the router
    real_estate_type = state["real_estate_type"]
    if real_estate_type == "Pass":
        return "Pass"
    else:
        return 'Fail'
    
def re_questions(state: RealEstateState) -> RealEstateState:
    print("=================================")
    print("""[re_questions] 질문이 부동산 관련이 아니거나 제대로 인식되지 않았습니다.
          부동산 관련 질문을 좀 더 자세하게 작성해주시면 답변드리겠습니다!!!""")
    new_question = input("새로운 부동산 질문을 입력해주세요: ")
    print("=================================")
    # 수정된 질문을 state에 업데이트
    return RealEstateState(messages=new_question)

In [21]:
import sqlite3
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool

def get_db_engine(db_path):
    """로컬 SQLite DB 파일과 연결된 엔진을 생성합니다."""
    try:
        # SQLite DB 파일과 연결
        connection = sqlite3.connect(db_path, check_same_thread=False)
        # SQLAlchemy 엔진 생성
        engine = create_engine(
            f"sqlite:///{db_path}",
            poolclass=StaticPool,
            connect_args={"check_same_thread": False}
        )
        return engine
    except Exception as e:
        print(f"데이터베이스 연결 중 오류 발생: {str(e)}")
        return None
    
    
# DB 파일 경로 지정
db_path = './data/real_estate_(1).db'
engine = get_db_engine(db_path)
db = SQLDatabase(
    engine,
    sample_rows_in_table_info=False  # 샘플 행 조회 비활성화
)

In [22]:
def generate_query(state: RealEstateState) -> RealEstateState:
    table = db.get_table_info(table_names=["addresses","sales", "rentals", "property_info", "property_locations","location_distances", "subway_stations"])
    prompt = f"""
    다음 데이터베이스 구조를 기반으로 사용자의 질문에 대한 SQL 쿼리를 생성해주세요:
    
    테이블 및 주요 컬럼 설명:
    {table}

    거리 및 대중교통 관련 계산 규칙:
    - **대중교통과 매물간의 거리 계산**: 테이블들을 활용해서 계산합니다.

    주의사항:
    1. 관련 조건은 SQL WHERE 절 형식으로 작성해주세요.
    2. '최근 5년' 같은 상대적 시간 표현은 현재 날짜 기준으로 변환해주세요.
    3. '추천할 만한', '인기 있는' 등의 표현은 적절히 해석하여 조건을 추가해주세요. 예: 'crime_rate < 5' 또는 'population_level = "높음"'.
    4. 텍스트 검색에는 LIKE 또는 MATCH ... AGAINST를 활용하세요.
    5. 여러 조건은 AND 또는 OR로 연결하세요.
    6. 정렬, 그룹화 등이 필요한 경우 이를 추가로 명시하세요. 예: ORDER BY created_at DESC.
    7. 쿼리만 작성하고 추가 설명은 하지 마세요.
    8. rental_type 값은 다음과 같습니다:
        - 'MONTHLY': 전세
        - 'YEARLY': 월세
    9. facilities는 영어로 쳐야지 나옵니다.
        - pi.facilities LIKE '%ELEVATOR%' -> 엘리베이터
        - pi.facilities LIKE '%aircon%' -> 에이컨, 에어콘
    10. 매물번호도 함께 보여주세요 매물번호는 property_id입니다.
    11. rentals 테이블에 price라는 컬럼이 없습니다. rentals 테이블에는 price 대신 deposit(보증금)과 monthly_rent(월세) 컬럼이 존재합니다.

    사용자 질문: {state['messages'][-1].content}

    SQL 쿼리 형식:
    SELECT * FROM table_name WHERE condition1 AND condition2 ...;
    """

    response = llm.invoke([
            SystemMessage(content="당신은 SQLite Database  쿼리를 생성하는 전문가입니다."),
            HumanMessage(prompt)
        ])
    
    return RealEstateState(query_sql=response.content)

In [23]:
def clean_sql_response(state: RealEstateState) -> RealEstateState:
    # 'query_sql' 키는 항상 존재한다고 가정
    query_sql = state['query_sql']

    # 코드 블록(````sql ... `````) 제거
    if query_sql.startswith("```sql") and query_sql.endswith("```"):
        query_sql = query_sql[6:-3].strip()  # "```sql" 제거 후 앞뒤 공백 제거

    # SQL 문 끝에 세미콜론 추가 (필요시)
    if not query_sql.strip().endswith(";"):
        query_sql = query_sql.strip() + ";"
        

    # 상태 업데이트
    return RealEstateState(query_sql=query_sql)

In [24]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

def run_query(state: RealEstateState) -> RealEstateState:
    
    tool = QuerySQLDataBaseTool(db=db)
    results = tool._run(state["query_sql"])

    if results == '':
        results = '결과값이 없습니다!!! 다시 질문해주세요!'
        return RealEstateState(results=results)

    return RealEstateState(results=results)

In [25]:
def generate_response(state: RealEstateState)-> RealEstateState:
    prompt = state['messages'][-1].content
    context = state['results']
    context += "위 매물들에 대한 추천과 함께, 각 부동산과 관련된 세계 지식과 흥미로운 사실을 추가해주세요."

    response = llm.invoke([
            SystemMessage(content="당신은 부동산산 추천 전문가이자 세계 지식을 갖춘 AI입니다. 주어진 정보와 세계 지식을 결합하여 사용자의 질문에 답변해주세요. 구분선 이후 간단한 추천이유도 적어줍니다."),
            HumanMessage(content= f"컨텍스트: {context}\n\n질문: {prompt}\n\n각 부동산에 대해 관련된 세계 지식과 흥미로운 사실을 추가해주세요.")
        ])
    
    output = response.content.strip()

    return RealEstateState(answers=output)

In [26]:
workflow.add_node("Filter Question", filter_node)
workflow.add_node('Re_Questions', re_questions)
workflow.add_node('Generate_Query', generate_query)
workflow.add_node('Clean_Sql_Response', clean_sql_response)
workflow.add_node('Run_Query', run_query)
workflow.add_node('Generate_Response', generate_response)

In [27]:
workflow.add_conditional_edges(
    "Filter Question",
    fiter_router,
    { 'Pass': "Generate_Query", 'Fail': 'Re_Questions'}
)

In [28]:
workflow.add_edge(START, "Filter Question")
workflow.add_edge("Re_Questions", "Filter Question")
workflow.add_edge("Generate_Query", "Clean_Sql_Response")
workflow.add_edge("Clean_Sql_Response", "Run_Query")
workflow.add_edge("Run_Query", "Generate_Response")
workflow.add_edge("Generate_Response", END)

In [29]:
app = workflow.compile()

In [30]:
result = app.invoke({'messages': '서울시 강남역에서 1000미터 이내 전세 10억 매물을 추천해줘'})

In [31]:
print(result['answers'])

현재 강남역에서 1000미터 이내의 전세 10억 매물에 대한 구체적인 리스트를 제공할 수는 없지만, 인기 있는 지역 및 추천할 만한 특징을 소개할 수 있습니다. 

1. **역삼동**  
   서울 강남구의 한 부분으로, 강남역과 가까운 인프라가 잘 갖춰져 있습니다. 대기업 및 스타트업의 많은 사무실이 이곳에 위치해 있어 직장인들이 선호하는 전세 주택이 많습니다.  
   **세계 지식/사실**: 역삼동은 '역삼 그린 스퀘어'와 같은 근린공원과 함께, 지속 가능한 도시 개발의 좋은 사례로 주목받고 있습니다. 최근 몇 년간 이 지역에는 여러 유망 스타트업이 자리잡고 있습니다.

2. **삼성동**  
   삼성동은 코엑스, 스타필드와 같은 대형 복합쇼핑몰 및 문화 공간이 있는 곳으로, 재벌기업 HQ와 은행들이 모여 있어 상업적으로도 아주 활발한 지역입니다.  
   **세계 지식/사실**: 삼성동은 아시아의 금융허브로 성장 중이며, 2021년 하반기에는 한국판 뉴딜 정책으로 인해 많은 투자와 관련 프로젝트가 예정되어 있습니다. 

3. **신사동**  
   강남의 트렌디한 지역으로, 여러 유명 카페와 레스토랑, 패션 매장이 밀집해 있습니다. 예술과 패션의 중심지로, 젊은층과 외국인들에게 특히 인기가 많습니다.  
   **세계 지식/사실**: 신사동 가로수길은 서울의 패션과 트렌드를 선도하는 거리로, 매년 많은 관광객이 방문하는 명소로 자리잡고 있습니다.

4. **청담동**  
   고급 아파트와 명품 브랜드 매장이 많으며, 외국 대사관들이 위치한 지역으로 주거 환경이 우수합니다.  
   **세계 지식/사실**: 청담동은 한국의 '히든 자산'으로 불리며, 종종 세계적인 리얼 에스테이트 투자자들이 주목하는 지역입니다.

이런 지역들을 고려한다면 원하는 전세 매물을 쉽게 찾을 수 있을 것입니다. 각 지역의 특성을 잘 이해하고, 본인의 생활 스타일과 맞는 곳을 선택하는 것이 중요합니다.


In [32]:
print(result)

{'real_estate_type': 'Pass', 'messages': [HumanMessage(content='서울시 강남역에서 1000미터 이내 전세 10억 매물을 추천해줘', additional_kwargs={}, response_metadata={}, id='052d969f-da8f-4266-83ee-8625d3ca03a2')], 'query_sql': "SELECT pi.property_id, pi.building_name, pi.detail_address, r.deposit, r.monthly_rent \nFROM property_info AS pi\nJOIN rentals AS r ON pi.property_id = r.property_id\nJOIN property_locations AS pl ON pi.property_location_id = pl.property_location_id\nJOIN subway_stations AS ss ON ss.address_id = pl.property_location_id\nJOIN location_distances AS ld ON ld.property_location_id = pl.property_location_id\nWHERE ss.line_info LIKE '%강남역%'\nAND ld.distance <= 1000\nAND r.rental_type = 'MONTHLY'\nAND r.deposit <= 1000000000\nORDER BY pi.first_seen DESC;", 'results': '결과값이 없습니다!!! 다시 질문해주세요!', 'answers': "현재 강남역에서 1000미터 이내의 전세 10억 매물에 대한 구체적인 리스트를 제공할 수는 없지만, 인기 있는 지역 및 추천할 만한 특징을 소개할 수 있습니다. \n\n1. **역삼동**  \n   서울 강남구의 한 부분으로, 강남역과 가까운 인프라가 잘 갖춰져 있습니다. 대기업 및 스타트업의 많은 사무실이 이곳에 위치해 있어 직장인들이 

In [33]:
print(result['query_sql'])

SELECT pi.property_id, pi.building_name, pi.detail_address, r.deposit, r.monthly_rent 
FROM property_info AS pi
JOIN rentals AS r ON pi.property_id = r.property_id
JOIN property_locations AS pl ON pi.property_location_id = pl.property_location_id
JOIN subway_stations AS ss ON ss.address_id = pl.property_location_id
JOIN location_distances AS ld ON ld.property_location_id = pl.property_location_id
WHERE ss.line_info LIKE '%강남역%'
AND ld.distance <= 1000
AND r.rental_type = 'MONTHLY'
AND r.deposit <= 1000000000
ORDER BY pi.first_seen DESC;


In [92]:
from typing import TypedDict, Annotated, List, Dict
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_openai.chat_models.base import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

from dotenv import load_dotenv
from langsmith import Client
import json

import sqlite3
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool

load_dotenv() 
client = Client() # langsmith 추적

class RealEstateState(TypedDict): # 그래프의 상태를 정의하는 클래스
    real_estate_type: Annotated[str ,"부동산 유형 (예: 아파트, 상가)"]
    keywordlist: Annotated[List[Dict] ,"키워드 리스트"]
    messages: Annotated[list, add_messages]
    query_sql: Annotated[str ,"생성된 SQL 쿼리"]
    results: Annotated[List[Dict], "쿼리 결과"]
    answers: Annotated[List[str], "최종 답변 결과"]
    query_answer:Annotated[str, 'answer다듬기']

# Step 1: LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=1)

# Step 2: StateGraph 정의
workflow = StateGraph(RealEstateState)

def filter_node(state:RealEstateState) -> RealEstateState:
    system_prompt = """
    Classify if a given question is related to real estate. If the question is related to topics such as property transactions, rental conditions, location recommendations, or property features, return Pass. If it's not directly related to real estate, return Fail.

    # Output Format
    - Return Pass if the question is real estate-related, otherwise return Fail.

    # Examples
    - 입력: '서울 아파트 매매 가격이 어떻게 되나요?'
      출력: Pass

    - 입력: '이 동네 전세 시세 알려주세요'
      출력: Pass

    - 입력: '서울에서 월세 계약 조건이 어떻게 되나요?'
      출력: Pass

    - 입력: '서울 아파트 매매가 얼마인가요?'
      출력: Pass

    - 입력: '대치동에서 버스정류장과 지하철이 가장 가까운 곳으로 알려줘'
      출력: Pass

    - 입력: '여자 혼자 살기 좋은 곳 추천해줘'
      출력: Pass

    - 입력: '교통이 편리하고 저렴한 원룸 추천해줘'
      출력: Pass

    - 입력: '이 음식점이 맛있나요?'
      출력: Fail

    - 입력: '서울에서 가장 큰 공원이 어디인가요?'
      출력: Fail
    """
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(state["messages"][-1].content)
    ])

    real_estate_type = response.content.strip()
    return RealEstateState(real_estate_type=real_estate_type)

def fiter_router(state: RealEstateState):
    # This is the router
    real_estate_type = state["real_estate_type"]
    if real_estate_type == "Pass":
        return "Pass"
    else:
        return 'Fail'
    
def re_questions(state: RealEstateState) -> RealEstateState:
    print("=================================")
    print("""[re_questions] 질문이 부동산 관련이 아니거나 제대로 인식되지 않았습니다.
          부동산 관련 질문을 좀 더 자세하게 작성해주시면 답변드리겠습니다!!!""")
    new_question = input("새로운 부동산 질문을 입력해주세요: ")
    print("=================================")
    # 수정된 질문을 state에 업데이트
    return RealEstateState(messages=new_question)

import sqlite3
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool

def get_db_engine(db_path):
    """로컬 SQLite DB 파일과 연결된 엔진을 생성합니다."""
    try:
        # SQLite DB 파일과 연결
        connection = sqlite3.connect(db_path, check_same_thread=False)
        # SQLAlchemy 엔진 생성
        engine = create_engine(
            f"sqlite:///{db_path}",
            poolclass=StaticPool,
            connect_args={"check_same_thread": False}
        )
        return engine
    except Exception as e:
        print(f"데이터베이스 연결 중 오류 발생: {str(e)}")
        return None
    
    
# DB 파일 경로 지정
db_path = './data/real_estate_(1).db'
engine = get_db_engine(db_path)
db = SQLDatabase(
    engine,
    sample_rows_in_table_info=False  # 샘플 행 조회 비활성화
)

def generate_query(state: RealEstateState) -> RealEstateState:
    table = db.get_table_info(table_names=["addresses","sales", "rentals", "property_info", "property_locations","location_distances"])
    prompt = f"""
    다음 데이터베이스 구조를 기반으로 사용자의 질문에 대한 SQL 쿼리를 생성해주세요:
    
    테이블 및 주요 컬럼 설명:
    {table}

    거리 및 대중교통 관련 계산 규칙:
    - **대중교통과 매물간의 거리 계산**: 테이블들을 활용해서 계산합니다.

    주의사항:
    1. 관련 조건은 SQL WHERE 절 형식으로 작성해주세요.
    2. '최근 5년' 같은 상대적 시간 표현은 현재 날짜 기준으로 변환해주세요.
    3. '추천할 만한', '인기 있는' 등의 표현은 적절히 해석하여 조건을 추가해주세요. 예: 'crime_rate < 5' 또는 'population_level = "높음"'.
    4. 텍스트 검색에는 LIKE 또는 MATCH ... AGAINST를 활용하세요.
    5. 여러 조건은 AND 또는 OR로 연결하세요.
    6. 정렬, 그룹화 등이 필요한 경우 이를 추가로 명시하세요. 예: ORDER BY created_at DESC.
    7. 쿼리만 작성하고 추가 설명은 하지 마세요.
    8. rental_type 값은 다음과 같습니다:
        - 'MONTHLY': 전세
        - 'YEARLY': 월세

    9. facilities는 영어로 쳐야지 나옵니다.
        - pi.facilities LIKE '%ELEVATOR%' -> 엘리베이터
        - pi.facilities LIKE '%aircon%' -> 에이컨, 에어콘

    10. 매물번호도 함께 보여주세요 매물번호는 property_id입니다.

    11. rentals 테이블에 price라는 컬럼이 없습니다. rentals 테이블에는 price 대신 deposit(보증금)과 monthly_rent(월세) 컬럼이 존재합니다.
    
    12. description은 매물에 대한 설명이 적혀있습니다. 쿼리를 짤 때 무조건 포함 시키세요.

    사용자 질문: {state['messages'][-1].content}

    SQL 쿼리 형식:
    SELECT * FROM table_name WHERE condition1 AND condition2 ...;
    """

    response = llm.invoke([
            SystemMessage(content="당신은 SQLite Database  쿼리를 생성하는 전문가입니다."),
            HumanMessage(prompt)
        ])
    
    return RealEstateState(query_sql=response.content)

def clean_sql_response(state: RealEstateState) -> RealEstateState:
    # 'query_sql' 키는 항상 존재한다고 가정
    query_sql = state['query_sql']

    # 코드 블록(````sql ... `````) 제거
    if query_sql.startswith("```sql") and query_sql.endswith("```"):
        query_sql = query_sql[6:-3].strip()  # "```sql" 제거 후 앞뒤 공백 제거

    # SQL 문 끝에 세미콜론 추가 (필요시)
    if not query_sql.strip().endswith(";"):
        query_sql = query_sql.strip() + ";"
        

    # 상태 업데이트
    return RealEstateState(query_sql=query_sql)

from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

def run_query(state: RealEstateState) -> RealEstateState:
    
    tool = QuerySQLDataBaseTool(db=db)
    results = tool._run(state["query_sql"])

    if results == '':
        results = '결과값이 없습니다!!! 다시 질문해주세요!'
        return RealEstateState(results=results)

    return RealEstateState(results=results)

def generate_response(state: RealEstateState)-> RealEstateState:
    prompt = state['messages'][-1].content
    context = state['results']
    context += "위 매물들에 대한 추천과 함께, 각 부동산과 관련된 세계 지식과 흥미로운 사실을 추가해주세요."

    response = llm.invoke([
            SystemMessage(content="당신은 부동산 추천 전문가이자 세계 지식을 갖춘 AI입니다. 주어진 정보와 세계 지식을 결합하여 사용자의 질문에 답변해주세요. 답변 맨위에는 매물번호를 적어주세요. 구분선 이후 간단한 추천이유도 적어줍니다."),
            HumanMessage(content= f"컨텍스트: {context}\n\n질문: {prompt}\n\n각 부동산에 대해 관련된 세계 지식과 흥미로운 사실을 추가해주세요.")
        ])
    
    output = response.content.strip()

    return RealEstateState(answers=output)

workflow.add_node("Filter Question", filter_node)
workflow.add_node('Re_Questions', re_questions)
workflow.add_node('Generate_Query', generate_query)
workflow.add_node('Clean_Sql_Response', clean_sql_response)
workflow.add_node('Run_Query', run_query)
workflow.add_node('Generate_Response', generate_response)

workflow.add_conditional_edges(
    "Filter Question",
    fiter_router,
    { 'Pass': "Generate_Query", 'Fail': 'Re_Questions'}
)

workflow.add_edge(START, "Filter Question")
workflow.add_edge("Re_Questions", "Filter Question")
workflow.add_edge("Generate_Query", "Clean_Sql_Response")
workflow.add_edge("Clean_Sql_Response", "Run_Query")
workflow.add_edge("Run_Query", "Generate_Response")
workflow.add_edge("Generate_Response", END)

app = workflow.compile()

result = app.invoke({'messages': '서울시 강남역에서 1000미터 이내 전세 10억 매물을 추천해줘'})

In [95]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "1"},  # 스레드 ID 설정
    tags=["my-rag"],  # Tag
)
memory = MemorySaver()

In [101]:
from typing import TypedDict, Annotated, List, Dict
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_openai.chat_models.base import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from dotenv import load_dotenv
from langsmith import Client
import json
import sqlite3
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool

load_dotenv() 
client = Client()  # LangSmith tracking

class RealEstateState(TypedDict):
    real_estate_type: Annotated[str, "부동산 유형 (예: 아파트, 상가)"]
    keywordlist: Annotated[List[Dict], "키워드 리스트"]
    messages: Annotated[list, add_messages]
    query_sql: Annotated[str, "생성된 SQL 쿼리"]
    results: Annotated[List[Dict], "쿼리 결과"]
    answers: Annotated[List[str], "최종 답변 결과"]
    query_answer: Annotated[str, 'answer다듬기']

# Step 1: LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

# Step 2: StateGraph 정의
workflow = StateGraph(RealEstateState)

# Stream utility
import sys

def stream_output(message):
    sys.stdout.write(message + "\n")
    sys.stdout.flush()

def filter_node(state: RealEstateState) -> RealEstateState:
    stream_output("[Filter Node] AI가 질문을 식별중입니다!!!!")
    system_prompt = """
    Classify if a given question is related to real estate. If the question is related to topics such as property transactions, rental conditions, location recommendations, or property features, return Pass. If it's not directly related to real estate, return Fail.

    # Output Format
    - Return Pass if the question is real estate-related, otherwise return Fail.

    # Examples
    - 입력: '서울 아파트 매매 가격이 어떻게 되나요?'
      출력: Pass

    - 입력: '이 동네 전세 시세 알려주세요'
      출력: Pass

    - 입력: '서울에서 월세 계약 조건이 어떻게 되나요?'
      출력: Pass

    - 입력: '서울 아파트 매매가 얼마인가요?'
      출력: Pass

    - 입력: '대치동에서 버스정류장과 지하철이 가장 가까운 곳으로 알려줘'
      출력: Pass

    - 입력: '여자 혼자 살기 좋은 곳 추천해줘'
      출력: Pass

    - 입력: '교통이 편리하고 저렴한 원룸 추천해줘'
      출력: Pass

    - 입력: '이 음식점이 맛있나요?'
      출력: Fail

    - 입력: '서울에서 가장 큰 공원이 어디인가요?'
      출력: Fail
    """
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(state["messages"][-1].content)
    ])

    real_estate_type = response.content.strip()
    stream_output(f"[Filter Node] AI가 질문을 식별했습니다.: {real_estate_type}")
    return RealEstateState(real_estate_type=real_estate_type)

def fiter_router(state: RealEstateState):
    # This is the router
    real_estate_type = state["real_estate_type"]
    if real_estate_type == "Pass":
        return "Pass"
    else:
        return 'Fail'

def re_questions(state: RealEstateState) -> RealEstateState:
    stream_output("=================================")
    stream_output("""[re_questions] 질문이 부동산 관련이 아니거나 제대로 인식되지 않았습니다.
          부동산 관련 질문을 좀 더 자세하게 작성해주시면 답변드리겠습니다!!!""")
    new_question = input("새로운 부동산 질문을 입력해주세요: ")
    stream_output("=================================")
    return RealEstateState(messages=new_question)

def get_db_engine(db_path):
    """로컬 SQLite DB 파일과 연결된 엔진을 생성합니다."""
    try:
        # SQLite DB 파일과 연결
        connection = sqlite3.connect(db_path, check_same_thread=False)
        # SQLAlchemy 엔진 생성
        engine = create_engine(
            f"sqlite:///{db_path}",
            poolclass=StaticPool,
            connect_args={"check_same_thread": False}
        )
        return engine
    except Exception as e:
        print(f"데이터베이스 연결 중 오류 발생: {str(e)}")
        return None
    
    
# DB 파일 경로 지정
db_path = './data/real_estate_(1).db'
engine = get_db_engine(db_path)
db = SQLDatabase(
    engine,
    sample_rows_in_table_info=False  # 샘플 행 조회 비활성화
)

def generate_query(state: RealEstateState) -> RealEstateState:
    stream_output("[Generate Query] AI가 쿼리를 생성중입니다...")
    
    table = db.get_table_info(table_names=["addresses", "sales", "rentals", "property_info", "property_locations", "location_distances", "subway_stations"])
    prompt = f"""
    다음 데이터베이스 구조를 기반으로 사용자의 질문에 대한 SQL 쿼리를 생성해주세요:
    
    테이블 및 주요 컬럼 설명:
    {table}

    거리 및 대중교통 관련 계산 규칙:
    - **대중교통과 매물간의 거리 계산**: 테이블들을 활용해서 계산합니다.

    주의사항:
    1. 관련 조건은 SQL WHERE 절 형식으로 작성해주세요.
    2. '최근 5년' 같은 상대적 시간 표현은 현재 날짜 기준으로 변환해주세요.
    3. '추천할 만한', '인기 있는' 등의 표현은 적절히 해석하여 조건을 추가해주세요. 예: 'crime_rate < 5' 또는 'population_level = "높음"'.
    4. 텍스트 검색에는 LIKE 또는 MATCH ... AGAINST를 활용하세요.
    5. 여러 조건은 AND 또는 OR로 연결하세요.
    6. 정렬, 그룹화 등이 필요한 경우 이를 추가로 명시하세요. 예: ORDER BY created_at DESC.
    7. 쿼리만 작성하고 추가 설명은 하지 마세요.
    8. rental_type 값은 다음과 같습니다:
        - 'MONTHLY': 전세
        - 'YEARLY': 월세
    9. facilities는 영어로 쳐야지 나옵니다.
        - pi.facilities LIKE '%ELEVATOR%' -> 엘리베이터
        - pi.facilities LIKE '%aircon%' -> 에이컨, 에어콘
    10. property_info 테이블 안에 있는 칼럼인 property_id를 반드시 포함해주세요.
    11. rentals 테이블에 price라는 컬럼이 없습니다. rentals 테이블에는 price 대신 deposit(보증금)과 monthly_rent(월세) 컬럼이 존재합니다.
    12. property_info 테이블 안에 있는 칼럼인 description에는 매물에 대한 설명이 적혀있습니다. 반드시 포함해주세요.
    13. pl.sido에는 값이 '서울특별시'밖에 없습니다. 해당 칼럼은 사용하지마세요.
    
    14. 돈과 관련된 칼럼인 price, deposit과 monthly_rent는 반드시 보여주세요.

    15. 쿼리문은 결과값이 5개만 나오게 해줘
    
    사용자 질문: {state['messages'][-1].content}

    SQL 쿼리 형식:
    SELECT * FROM table_name WHERE condition1 AND condition2 ...;
    """

    response = llm.invoke([
        SystemMessage(content="당신은 SQLite Database 쿼리를 생성하는 전문가입니다."),
        HumanMessage(prompt)
    ])

    stream_output("[Generate Query] AI가 데이터베이스 쿼리를 생성했습니다.")
    return RealEstateState(query_sql=response.content)

def clean_sql_response(state: RealEstateState) -> RealEstateState:
    stream_output("[Clean SQL Response] 쿼리를 이쁘게 만들고 있습니다...")
    query_sql = state['query_sql']

    if query_sql.startswith("```sql") and query_sql.endswith("```"):
        query_sql = query_sql[6:-3].strip()

    if not query_sql.strip().endswith(";"):
        query_sql = query_sql.strip() + ";"

    stream_output("[Clean SQL Response] 쿼리 정제가 끝났습니다.")
    return RealEstateState(query_sql=query_sql)

def run_query(state: RealEstateState) -> RealEstateState:
    stream_output("[Run Query] 쿼리 실행중...")
    tool = QuerySQLDataBaseTool(db=db)
    results = tool._run(state["query_sql"])

    if results == '':
        results = '쿼리 실행 결과값이 없습니다!'
        stream_output("[Run Query] 쿼리 실행 결과값이 없습니다!")
        return RealEstateState(results=results)

    stream_output("[Run Query] 쿼리 실행 성공.")
    return RealEstateState(results=results)

def generate_response(state: RealEstateState) -> RealEstateState:
    stream_output("[Generate Response] 거의 다 했어요...")
    prompt = state['messages'][-1].content
    context = state['results']
    # context += """
    #         위 매물들에 대한 추천과 함께 부동산 답변과 관련된 세계 지식과 흥미로운 사실을 추가해주세요.
    #         """

    response = llm.invoke([
        SystemMessage(content="""
                        당신은 부동산 추천 전문가입니다.
                        사용자 질문에 부동산 매물을 추천해주세요요
                        답변에는 오직 property_id와 간단한 추천이유만 적어줍니다.
                        """
                    ),
        HumanMessage(content= f"컨텍스트: {context}\n\n질문: {prompt}\n\n각 부동산에 대해 관련된 세계 지식과 흥미로운 사실을 추가해주세요.")
    ])

    output = response.content.strip()
    stream_output("[Generate Response] 답변 생성 끝.")
    return RealEstateState(answers=output)

workflow.add_node("Filter Question", filter_node)
workflow.add_node('Re_Questions', re_questions)
workflow.add_node('Generate_Query', generate_query)
workflow.add_node('Clean_Sql_Response', clean_sql_response)
workflow.add_node('Run_Query', run_query)
workflow.add_node('Generate_Response', generate_response)

workflow.add_conditional_edges(
    "Filter Question",
    fiter_router,
    { 'Pass': "Generate_Query", 'Fail': 'Re_Questions'}
)

workflow.add_edge(START, "Filter Question")
workflow.add_edge("Re_Questions", "Filter Question")
workflow.add_edge("Generate_Query", "Clean_Sql_Response")
workflow.add_edge("Clean_Sql_Response", "Run_Query")
workflow.add_edge("Run_Query", "Generate_Response")
workflow.add_edge("Generate_Response", END)

app = workflow.compile()
while True:
    user_input = input()
    if user_input == 'q':
        break
    result = app.invoke({'messages': user_input}, config=config)
    print(result['answers'])


[Filter Node] AI가 질문을 식별중입니다!!!!
[Filter Node] AI가 질문을 식별했습니다.: Pass
[Generate Query] AI가 쿼리를 생성중입니다...
[Generate Query] AI가 데이터베이스 쿼리를 생성했습니다.
[Clean SQL Response] 쿼리를 이쁘게 만들고 있습니다...
[Clean SQL Response] 쿼리 정제가 끝났습니다.
[Run Query] 쿼리 실행중...
[Run Query] 쿼리 실행 성공.
[Generate Response] 거의 다 했어요...
[Generate Response] 답변 생성 끝.
property_id: 6901968, 추천이유: 강서구에 위치하며, 실매물로 보증금이 2억 3천800만원으로 전세로 적합합니다.


In [94]:
print(result['results'])

[(6988759, None, None, '2월중 협의입주,올수리,착한가격좋은 전세임', 360000000, 0), (6987260, '위너스빌', '401호', '🏡신창 시장 1분 거리에 자리하는 \r\n빌라로  쌍문역과 수유역이 도보\r\n10분 거리입니다.\r\n\r\n💒신축 1년이  되었으며 엘리베이\r\n터가 있고 임대인이 임대사업자로서 \r\n보증보험 의무 가입자입니다.\r\n\r\n🏠내부는 방이 2개에  거실과 주방\r\n화장실, 다용도실이 구조 좋고 이쁜  \r\n집입니다.\r\n______________________________\r\n\r\n👩\u200d🦱수유동에서만 12년차이고 25년 \r\n경력의 부동산입니다. 좋은 집 구하실 \r\n분, 내놓으실 분 전화주세요📞\r\n\r\n🚘가나부동산 오시는 길 \r\n강북구 삼양로 512(수유동...', 240000000, 0), (6984864, None, '502호', "◈매물특징◈\r\n\r\n'- 방2개,욕실1개,거실전용넓고,작은 테라스와베란다있음,옥상사용가능,주차편리\r\n'- 생활시설 편리한 마트 2분거리,교통시설편리,주민센터 초등학교 매우 가까움\r\n'- 허그안심대출 협조 가능 하며 근저당 말소예정임\r\n'- 탑층으로 조용하고 층간 소음 걱정 없음\r\n'- 내관 외관 관리가 잘되어 있음\r\n'- 최근 올수리 하였음 도배,장판,싱크대,보일러,화장실하였고 2010년식이며 샷시도 잘되어 있음\r\n'- 공실로 즉시 입주 가능", 115000000, 0), (6983761, None, None, '솔밭공원역3분 버스정류장2분 솔밭공원맞은편 우이천계곡인접\r\n덕성여대와 백운초 서라벌중 효문중 고 인근에 위치\r\n전세대출 보증보험가능\r\n층간소음 걱정없는 어린 자녀 키우기 좋은\r\n편리하고 넓은 지하주차장\r\n인근에 도서관 은행 시장 마트 병의원 약국 등 인프라 좋음\r\n\r\n찾아 오시는길\r\n\r\n강북구 4.19로 9, 아담공인중개사사무소\r\n010-3668-8122\r